In [1]:
import pandas as pd
import numpy as np
import os
import datetime

In [2]:
path_excel_ta="/home/jian/Projects/Big_Lots/New_TA/package_to_run_TA_every_quarter/output_2020-07-22/BL_final_TA_updated_JL_2020-07-22.xlsx"
path_SOTF_dom="/home/jian/BigLots/static_files/store_list_from_Dom/Store List Report 07.10.20 425PM.xlsx"


In [3]:
excel=pd.ExcelFile(path_excel_ta)
excel.sheet_names

['unique_zips_full_footprint',
 'view_by_TA',
 'view_by_store',
 'view_for_Tableau']

In [4]:
df_zip_by_store=excel.parse("view_for_Tableau",dtype=str)

df_zip_by_store.head(5)

,TA_num,location_id,ta_name,zip_cd,zip_type,latitude_meas,longitude_meas
0,1,4526,multiple_1,76248,trans_P,32.933601,-97.217728
1,1,4526,multiple_1,76244,trans_P,32.933601,-97.217728
2,1,4526,multiple_1,76182,trans_S,32.933601,-97.217728
3,1,4526,multiple_1,76034,trans_S,32.933601,-97.217728
4,1,4526,multiple_1,76092,trans_S,32.933601,-97.217728


In [6]:
df_store_type=pd.ExcelFile(path_SOTF_dom)
df_store_type=df_store_type.parse("Store List",dtype=str)
df_store_type=df_store_type[["Store",'SOTF']]
df_store_type=df_store_type[df_store_type['SOTF'].str.lower()=="yes"]
df_store_type=df_store_type[df_store_type['Store']!="TBD"]
print(len(df_store_type),df_store_type['Store'].nunique())
df_store_type=df_store_type.rename(columns={"Store":"location_id","SOTF":"Store_Type"})
df_store_type['Store_Type']="SOTF"
df_store_type.head(2)

491 491


,location_id,Store_Type
0,1,SOTF
7,49,SOTF


In [8]:
df_zip_by_store=pd.merge(df_zip_by_store,df_store_type,on="location_id",how="left")
df_zip_by_store['Store_Type']=df_zip_by_store['Store_Type'].fillna("Legacy")

In [10]:
print(df_zip_by_store['zip_type'].unique())

['trans_P' 'trans_S' 'zips_10']


In [12]:
writer=pd.ExcelWriter("./BL_zip_lable_by_store_type_JL_%s.xlsx"%str(datetime.datetime.now().date()),engine="xlsxwriter")
for store_type, df_group in df_zip_by_store.groupby("Store_Type"):
    df=df_group[['Store_Type','zip_cd','zip_type']].drop_duplicates()
    df['zip_type']=df['zip_type'].replace("trans_P","P").replace("trans_S","S").replace("zips_10","P")
    df=df.sort_values(["zip_cd","zip_type"]).drop_duplicates("zip_cd")
    df.to_excel(writer,store_type,index=False)
    

In [14]:
df_both_zip_lable=excel.parse("unique_zips_full_footprint",dtype=str)
df_both_zip_lable['zip_type']=df_both_zip_lable['zip_type'].replace("zips_10","trans_P")
df_both_zip_lable['zip_type']=df_both_zip_lable['zip_type'].apply(lambda x: x.split("_")[1])
df_both_zip_lable.to_excel(writer,"unique_zips_full_footprint",index=False)
writer.save()